# LangGraph 진로 상담 시스템

이 노트북에서는 LangGraph를 사용하여 고등학생을 위한 진로 상담 시스템을 구현합니다.

## 목표
- 7단계로 구성된 체계적인 진로 탐색 과정
- AI 기반 개인화된 진로 상담
- 2022 개정 교육과정과 연계된 실천 활동 제시

## 구성
1. 라이브러리 설정 및 초기화
2. 상태 정의 및 시스템 설정
3. 각 단계별 함수 구현
4. 그래프 구성 및 실행

In [28]:
# 필요한 라이브러리 import
from typing import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
import os

In [29]:
# 진로 상담 상태 정의
class CareerState(TypedDict):
    """진로 상담 시스템의 상태를 정의하는 클래스"""
    user_input: str           # 사용자 입력
    career_choice: str        # 선택한 직업
    current_response: str     # 현재 응답 메시지
    is_complete: bool         # 완료 여부

In [30]:
# 직업 입력 노드 함수
def career_input_node(state: CareerState) -> CareerState:
    """사용자로부터 희망 직업을 입력받는 노드"""
    
    # 처음 시작하는 경우 (사용자 입력이 없는 경우)
    if not state.get("user_input"):
        response = """
🎓 안녕하세요! 진로 상담사입니다.

당신의 꿈과 목표를 찾아가는 여정을 함께 시작해보겠습니다!

**질문: 미래에 어떤 직업을 갖고 싶으신가요?**

예시:
- 의사, 교사, 소프트웨어 개발자
- 디자이너, 유튜버, 심리상담사
- 건축가, 요리사, 항공기 조종사 등

자유롭게 입력해주세요! 🌟
        """
        
        return {
            **state,
            "current_response": response.strip(),
            "is_complete": False
        }
    
    # 사용자가 직업을 입력한 경우
    else:
        career_choice = state["user_input"].strip()
        
        if career_choice:
            response = f"""
훌륭한 선택입니다! 🎉

**선택하신 직업:** {career_choice}

{career_choice}는 정말 의미있는 직업이네요! 
이 직업을 통해 많은 사람들에게 도움을 줄 수 있을 것 같습니다.

앞으로 더 구체적인 진로 계획을 세워나가겠습니다!
            """
            
            return {
                **state,
                "career_choice": career_choice,
                "current_response": response.strip(),
                "is_complete": True  # 현재는 한 단계만 구현하므로 완료 처리
            }
        else:
            return {
                **state,
                "current_response": "직업명을 입력해주세요! 어떤 직업에 관심이 있으신가요? 😊",
                "is_complete": False
            }

In [34]:
# 그래프 생성 및 구성 (수정된 버전)
def create_career_graph():
    """진로 상담 그래프를 생성하는 함수"""
    
    # StateGraph 생성
    workflow = StateGraph(CareerState)
    
    # 노드 추가
    workflow.add_node("career_input", career_input_node)
    
    # 시작점 설정
    workflow.set_entry_point("career_input")
    
    # 단순한 엣지 설정: career_input에서 바로 END로
    workflow.add_edge("career_input", END)
    
    # 메모리 세이버 추가
    memory = MemorySaver()
    
    # 그래프 컴파일 (재귀 제한 설정)
    app = workflow.compile(
        checkpointer=memory,
        interrupt_before=[],  # 중단점 없음
        interrupt_after=[]    # 중단점 없음
    )
    
    return app

In [35]:
# 그래프 시각화
def visualize_graph():
    """그래프 구조를 시각화하는 함수"""
    
    try:
        # 그래프 생성
        app = create_career_graph()
        
        # 그래프 구조를 mermaid 형식으로 출력
        print("📊 진로 상담 그래프 구조:")
        print("="*50)
        print(app.get_graph().draw_mermaid())
        print("="*50)
        
        # 그래프 정보 출력
        print("\n🔍 그래프 상세 정보:")
        print(f"- 노드 수: {len(app.get_graph().nodes)}")
        print(f"- 엣지 수: {len(app.get_graph().edges)}")
        print(f"- 시작점: {app.get_graph().first_node}")
        print(f"- 종료점: END")
        
        print("\n📝 노드 목록:")
        for node in app.get_graph().nodes:
            print(f"  - {node}")
            
        return app
        
    except Exception as e:
        print(f"❌ 그래프 시각화 중 오류 발생: {e}")
        return None

# 그래프 생성 및 시각화 실행
app = visualize_graph()

📊 진로 상담 그래프 구조:
%%{init: {'flowchart': {'curve': 'linear'}}}%%
graph TD;
	__start__([<p>__start__</p>]):::first
	career_input(career_input)
	__end__([<p>__end__</p>]):::last
	__start__ --> career_input;
	career_input --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc


🔍 그래프 상세 정보:
- 노드 수: 3
- 엣지 수: 2
- 시작점: <bound method Graph.first_node of Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=<class 'langchain_core.utils.pydantic.LangGraphInput'>, metadata=None), 'career_input': Node(id='career_input', name='career_input', data=career_input(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), '__end__': Node(id='__end__', name='__end__', data=<class 'langchain_core.utils.pydantic.LangGraphOutput'>, metadata=None)}, edges=[Edge(source='__start__', target='career_input', data=None, conditional=False), Edge(source='career_input', target='__end__', data

In [36]:
# 수정된 테스트 실행
def test_career_input():
    """진로 입력 기능을 테스트하는 함수"""
    
    # 새로운 그래프 생성
    app = create_career_graph()
    
    print("\n🧪 진로 입력 테스트 시작...")
    
    # 테스트용 설정 (재귀 제한 추가)
    thread_config = {
        "configurable": {
            "thread_id": "test_session"
        },
        "recursion_limit": 50  # 재귀 제한 증가
    }
    
    try:
        # 1단계: 환영 메시지 출력 (사용자 입력 없음)
        print("\n--- 1단계: 환영 메시지 ---")
        initial_state = {
            "user_input": "",
            "career_choice": "",
            "current_response": "",
            "is_complete": False
        }
        
        result = app.invoke(initial_state, config=thread_config)
        print("🤖 상담가:", result["current_response"])
        print(f"완료 상태: {result.get('is_complete', False)}")
        
        # 2단계: 직업 입력 시뮬레이션
        print("\n--- 2단계: 직업 입력 ('소프트웨어 개발자') ---")
        next_state = {
            "user_input": "소프트웨어 개발자",
            "career_choice": "",
            "current_response": "",
            "is_complete": False
        }
        
        final_result = app.invoke(next_state, config=thread_config)
        print("🤖 상담가:", final_result["current_response"])
        print(f"\n✅ 선택된 직업: {final_result.get('career_choice', '없음')}")
        print(f"✅ 완료 상태: {final_result.get('is_complete', False)}")
        
        return final_result
        
    except Exception as e:
        print(f"❌ 테스트 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
        return None

# 테스트 실행
print("\n" + "="*60)
print("🚀 수정된 진로 상담 시스템 테스트")
print("="*60)

test_result = test_career_input()


🚀 수정된 진로 상담 시스템 테스트

🧪 진로 입력 테스트 시작...

--- 1단계: 환영 메시지 ---
🤖 상담가: 🎓 안녕하세요! 진로 상담사입니다.

당신의 꿈과 목표를 찾아가는 여정을 함께 시작해보겠습니다!

**질문: 미래에 어떤 직업을 갖고 싶으신가요?**

예시:
- 의사, 교사, 소프트웨어 개발자
- 디자이너, 유튜버, 심리상담사
- 건축가, 요리사, 항공기 조종사 등

자유롭게 입력해주세요! 🌟
완료 상태: False

--- 2단계: 직업 입력 ('소프트웨어 개발자') ---
🤖 상담가: 훌륭한 선택입니다! 🎉

**선택하신 직업:** 소프트웨어 개발자

소프트웨어 개발자는 정말 의미있는 직업이네요! 
이 직업을 통해 많은 사람들에게 도움을 줄 수 있을 것 같습니다.

앞으로 더 구체적인 진로 계획을 세워나가겠습니다!

✅ 선택된 직업: 소프트웨어 개발자
✅ 완료 상태: True
